## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [29]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [30]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [31]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [32]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210202.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [33]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,324180,6800,330980,313286,0.946541,90764,2021-02-01
1,Aragón,55305,1200,56505,56482,0.999593,11119,2021-02-01
2,Asturias,58235,1200,59435,58177,0.978834,19503,2021-02-01
3,Baleares,29610,600,30210,25963,0.859417,5026,2021-02-01
4,Canarias,64320,1300,65620,67783,1.032963,14647,2021-02-01
5,Cantabria,27645,500,28145,25067,0.890638,4261,2021-02-01
6,Castilla y Leon,121395,2500,123895,126362,1.019912,35051,2021-02-01
7,Castilla La Mancha,82935,1700,84635,88888,1.050251,21360,2021-02-01
8,Cataluña,280250,5800,286050,261562,0.914393,69431,2021-02-01
9,C. Valenciana,142930,2900,145830,147945,1.014503,44979,2021-02-01


..hacemos una copia para hacerle las transformaciones necesarias:

In [34]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [35]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [36]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [37]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [38]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [39]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                     'Dosis entregadas Moderna','Dosis entregadas',
                     'Dosis administradas','% sobre entregadas',
                     'Total pauta completada',
                     'Fecha de la ultima vacuna registrada',
                     'source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [40]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [41]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [42]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,2/2/21,Andalucía,324180,6800,330980,313286,0.946541,90764,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,2/2/21,Aragón,55305,1200,56505,56482,0.999593,11119,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,2/2/21,Asturias,58235,1200,59435,58177,0.978834,19503,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,2/2/21,Baleares,29610,600,30210,25963,0.859417,5026,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,2/2/21,Canarias,64320,1300,65620,67783,1.032963,14647,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,2/2/21,Cantabria,27645,500,28145,25067,0.890638,4261,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,2/2/21,Castilla y Leon,121395,2500,123895,126362,1.019912,35051,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,2/2/21,Castilla La Mancha,82935,1700,84635,88888,1.050251,21360,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,2/2/21,Cataluña,280250,5800,286050,261562,0.914393,69431,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,2/2/21,C. Valenciana,142930,2900,145830,147945,1.014503,44979,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [43]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [44]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [45]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...
395,1/2/21,Navarra,27645.0,500.0,28145,25116,0.892379,5281.0,NaN,31/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
396,1/2/21,País Vasco,74700.0,1400.0,76100,57262,0.752457,12277.0,NaN,31/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
397,1/2/21,Ceuta,1980.0,200.0,2180,1890,0.866972,688.0,NaN,29/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
398,1/2/21,Melilla,1980.0,200.0,2180,2113,0.969266,806.0,NaN,29/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [46]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [47]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas', 'Dosis administradas',
       '% sobre entregadas', 'Total pauta completada',
       'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [48]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [49]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...
415,2/2/21,Navarra,27645.0,500.0,28145,26779,0.951466,6320.0,NaN,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
416,2/2/21,País Vasco,74700.0,1400.0,76100,59582,0.782943,14059.0,NaN,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
417,2/2/21,Ceuta,1980.0,200.0,2180,1950,0.894495,748.0,NaN,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
418,2/2/21,Melilla,1980.0,200.0,2180,2348,1.077064,1040.0,NaN,1/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [50]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [51]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)